In [1]:
import csv
import copy
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
csv_file = "data/stock_clean.csv"
df = pd.read_csv(csv_file)

In [3]:
df = df.rename(columns = {df.columns[0]:"date"})

In [4]:
df["date"]

0      19620831
1      19620928
2      19621031
3      19621130
4      19621231
         ...   
391    19950331
392    19950428
393    19950531
394    19950630
395    19950731
Name: date, Length: 396, dtype: int64

In [5]:
result = df["date"].isin(["19720731"])
t = df["date"][result].index.values[0]+1
print(t)

120


In [6]:
#Drop columns that have less than pre 120 + post 12 = 132 valid return
tmp_df_all = df[t-119:t+13].dropna(thresh=df[t-119:t+13].shape[0],how='all',axis=1)
#print(tmp_df)
#Get the prev
tmp_df = tmp_df_all[0:120]
N = len(tmp_df.columns[1:])
q = 0.2
print(q)
print(N)

0.2
973


In [7]:
cov_matrix = tmp_df[tmp_df.columns[1:]].astype(np.float64).cov()
print(cov_matrix)

          ACF        AJ       ADX        AB        AN       AYP       ACD  \
ACF  0.005819  0.002265  0.001461  0.003777  0.002224  0.001158  0.001840   
AJ   0.002265  0.015953  0.001637  0.005977  0.002482  0.000411  0.002361   
ADX  0.001461  0.001637  0.002294  0.002777  0.001567  0.000986  0.001416   
AB   0.003777  0.005977  0.002777  0.012609  0.004451  0.001173  0.003906   
AN   0.002224  0.002482  0.001567  0.004451  0.006156  0.001159  0.003402   
..        ...       ...       ...       ...       ...       ...       ...   
WPP  0.000995  0.003838  0.001107  0.004381  0.001637  0.000677  0.001749   
WRO  0.001519  0.008800  0.001201  0.005405  0.002499  0.000720  0.001466   
WMH  0.002298  0.005013  0.002061  0.006612  0.001899  0.000677  0.001195   
WNW  0.001897  0.004548  0.000769  0.005400  0.002668  0.000137  0.003211   
WRT  0.000354  0.007894  0.000725  0.001269  0.000341 -0.000259 -0.000515   

           AH       AMX        AC  ...       UFD       UVC       VLX  \
ACF

In [8]:
cov_matrix_inv = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
print(cov_matrix_inv)

          ACF        AJ       ADX        AB        AN       AYP       ACD  \
ACF  2.130508 -0.188419 -0.024210 -0.501656 -0.238469 -0.007295 -0.018286   
AJ  -0.188419  3.652066  0.000188 -0.417014 -0.326211 -0.162612  0.134312   
ADX -0.024210  0.000188  0.895820  0.177858  0.140921 -0.011471  0.032421   
AB  -0.501656 -0.417014  0.177858  1.861858  0.148362 -0.023324 -0.016235   
AN  -0.238469 -0.326211  0.140921  0.148362  1.888520 -0.170127  0.175599   
..        ...       ...       ...       ...       ...       ...       ...   
WPP -0.487066 -0.222572  0.057294  0.186328  0.344139  0.071300 -0.095498   
WRO  0.184243 -0.564430  0.026852  0.372633  0.065775  0.151168 -0.070321   
WMH  0.097443  0.223680 -0.000505  0.334013 -0.129424 -0.149213 -0.117622   
WNW -0.209546  0.098365 -0.127383  0.019101  0.290019  0.016479  0.438158   
WRT -0.271305  0.703274  0.083338  0.063249 -0.069688  0.015514 -0.422131   

           AH       AMX        AC  ...       UFD       UVC       VLX  \
ACF

In [9]:
mu = tmp_df[tmp_df.columns[1:]].mean()
print(mu)

ACF    0.015436
AJ     0.007694
ADX    0.008230
AB     0.014622
AN     0.004169
         ...   
WPP    0.007190
WRO    0.016500
WMH    0.022722
WNW    0.014760
WRT    0.013992
Length: 973, dtype: float64


In [10]:
A = np.ones((1,N))@cov_matrix_inv.values@np.ones((N,1))
print(A)

[[1080.32758509]]


In [11]:
B = np.ones((1,N))@cov_matrix_inv.values@mu.values
print(B)

[9.06822128]


In [12]:
C = mu.values.T@cov_matrix_inv.values@mu.values
print(C)

0.5192947817780915


In [13]:
w = (C-q*B)/(A*C-B*B)*cov_matrix_inv.values@np.ones((N,1))+(q*A-B)/(A*C-B*B)*cov_matrix_inv.values@mu.values[:,np.newaxis]
#w = cov_matrix_inv.values@np.ones((N,1))/(np.ones((1,N))@cov_matrix_inv.values@np.ones((N,1)))
print(np.sum(w))
print(mu@w)

0.9999999999999998
[0.2]


In [14]:
tmp_df_post = tmp_df_all[120:]
tmp_df_post = tmp_df_post[tmp_df_post.columns[1:]]
#print(tmp_df_post)
annual_return = (tmp_df_post.astype(np.float64)+1).prod()-1
#print(annual_return)
#print(w)
print(annual_return@w)

[-0.39278722]


In [15]:
annual_returns = []
for i in tqdm(range(t,384,12)):
    #tmp_df = df[i-119:i+1].dropna(thresh=df[i-119:i+1].shape[0],how='all',axis=1)
    #Drop columns that have less than pre 120 + post 12 = 132 valid return
    tmp_df_all = df[i-119:i+13].dropna(thresh=df[i-119:i+13].shape[0],how='all',axis=1)
    #print(tmp_df)
    #Get the prev
    tmp_df = tmp_df_all[0:120]
    N = len(tmp_df.columns[1:])
    cov_matrix = tmp_df[tmp_df.columns[1:]].astype(np.float64).cov()
    cov_matrix_inv = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    mu = tmp_df[tmp_df.columns[1:]].mean()
    q = 0.2*tmp_df[tmp_df.columns[1:]].astype(np.float64).mean().mean()
    #print(mu)
    A = np.ones((1,N))@cov_matrix_inv.values@np.ones((N,1))
    B = np.ones((1,N))@cov_matrix_inv.values@mu.values
    C = mu.values.T@cov_matrix_inv.values@mu.values
    #w = (C-q*B)/(A*C-B*B)*cov_matrix_inv.values@np.ones((N,1))+(q*A-B)/(A*C-B*B)*cov_matrix_inv.values@mu.values[:,np.newaxis]
    w = cov_matrix_inv.values@np.ones((N,1))/(np.ones((1,N))@cov_matrix_inv.values@np.ones((N,1)))
    for j in range(12):
        tmp_df_post = tmp_df_all[120+j:121+j]
        tmp_df_post = tmp_df_post[tmp_df_post.columns[1:]]
        #print(tmp_df_post)
        annual_return = (tmp_df_post.astype(np.float64)+1).prod(axis=0)-1
        annual_returns.append(annual_return@w)
    #print(annual_return[0:10])
    print(np.sum(w))
    print(np.min(w))
    print(mu@w)
#     tmp_df_post = tmp_df_all[120:132]
#     tmp_df_post = tmp_df_post[tmp_df_post.columns[1:]]
#     annual_return = (tmp_df_post.astype(np.float64)+1).prod(axis=0)-1
#     annual_returns.append(annual_return@w)

  5%|▍         | 1/22 [00:00<00:05,  3.72it/s]

1.0
-0.01670403439180711
[0.00839396]


  9%|▉         | 2/22 [00:00<00:05,  3.59it/s]

1.0000000000000002
-0.01204275833353318
[0.00870321]


 14%|█▎        | 3/22 [00:00<00:05,  3.41it/s]

...

1.0
-0.009382960686887187
[0.01506685]


 95%|█████████▌| 21/22 [00:08<00:00,  3.33it/s]

1.0
-0.00972217650460845
[0.01587931]


100%|██████████| 22/22 [00:08<00:00,  2.50it/s]

0.9999999999999997
-0.00868113845028335
[0.0133736]


In [16]:
annual_returns = np.array(annual_returns)

In [17]:
annual_returns.std()*np.sqrt(12)

0.12205701484968583

In [18]:
annual_returns.mean()

0.011052453748610837